In [1]:
using StanModels
using CmdStan, StanMCMCChain

ProjDir = rel_path_s("..", "scripts", "12")
cd(ProjDir)

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;
first(d[[:culture, :population, :log_pop, :society]], 5)

m12_6 = "
data {
    int N;
    int N_societies;
    int total_tools[N];
    real logpop[N];
    int society[N];
}
parameters{
    real a;
    real bp;
    vector[N_societies] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[N_societies] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1),"N_societies" => 10,
"total_tools" => d[:total_tools], "logpop" => d[:log_pop],
"society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD        Naive SE         MCSE         ESS    
         lp__  927.828355000  3.14161435 0.049673284361 0.11391184934  760.620346
accept_stat__    0.937357280  0.08607389 0.001360947737 0.00186926599 1000.000000
   stepsize__    0.055005450  0.00518057 0.000081911998 0.00082945135   39.009752
  treedepth__    5.845250000  0.46835593 0.007405357466 0.01304325889 1000.000000
 n_leapfrog__   69.212000000 30.90504549 0.488651674626 0.82257157743 1000.000000
  divergent__    0.000000000  0.00000000 0.000000000000 0.00000000000         NaN
     energy__ -921.364402500  4.05188405 0.064065912117 0.14456084492  785.620238
            a    1.131973004  0.77846283 0.012308578147 0.02325224831 1000.000000
           bp    0.257828020  0.08476864 0.001340309830 0.00248302904 1000.000000
  a_society.1   -0.209175907  0.24454095 0.003866531853 0.006775

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*